# BIDSifer & Dicom Conversion

In [ ]:
import os
import os.path as op
import json
import shutil
import tarfile
import subprocess as sp
from glob import glob
import dcmstack as ds
import dicom as dcm
import pandas as pd
import numpy as np
import shutil 

def convert_nifti(target_folder, destination):
    os.makedirs(destination, exist_ok=True)
    src_dcms = '{0}/resources/DICOM/files/'.format(target_folder)
    cmd = "dcm2niix -o {0} {1}".format(destination, src_dcms)
    sp.Popen(cmd, shell=True).wait()
    for i in glob(destination + "/*"):
        idx = i.index(".")
        shutil.move(i, destination + i[idx:])
    shutil.rmtree(destination)
    
def save_json(filename, data): 
    with open(filename, 'w') as fp:
        json.dump(data, fp, sort_keys=True, indent=4)
        
df = pd.read_csv('heuristics.csv')        

for tar_file in glob('/home/data/madlab/Mattfeld_vCAT/sourcedata/Mattfeld_REVL-000-vCAT-*-S1.tar.gz'):
    sub = os.path.basename(tar_file).split('_')[-1].split('-')[-2] #gets only subject ID
    print(sub)
    if os.path.exists('/home/data/madlab/Mattfeld_vCAT/dset/sub-{0}/'.format(sub)): #check to see if this Ss is new
        continue       
    os.makedirs('/scratch/madlab/bidsify_vCAT/vCAT-{0}/'.format(sub), exist_ok=True) #creates directories
    #unzipping the file without deletion into new directories
    sp.Popen('tar -xf {0} -C /scratch/madlab/bidsify_vCAT/vCAT-{1}'.format(tar_file, sub), shell=True).wait()
    curr_dir = '/scratch/madlab/bidsify_vCAT/vCAT-{0}/Mattfeld_REVL-000-vCAT-{0}-S1/scans/'.format(sub)
    all_dirs = [x for x in glob(curr_dir + "*") if not "setter" in x and not "32ch" in x and not "MPRG" in x]
    corr_scan = {} #contains only corrected scans 
    corr_idx = {} #contains indices of correct scans within all_dirs
    for idx, curr_scan in enumerate(all_dirs): #dealing with the inconsistent numbering of scans
        i = int(curr_scan.split("/")[-1].split("-")[0]) #get scan number
        for row, scan in enumerate(df['init_name']): #scan names in heuristics file
            if scan in curr_scan: #if one of the scans we care about
                if not scan in corr_scan or corr_scan[scan] < i: 
                    corr_idx[scan] = idx
                    corr_scan[scan] = i 
    temp = []
    for key in corr_scan:
        temp.append(all_dirs[corr_idx[key]])
    all_dirs = temp        
    for i in ['anat', 'func', 'dwi', 'fmap']:      
        os.makedirs('/home/data/madlab/Mattfeld_vCAT/dset/sub-{0}/{1}/'.format(sub, i), exist_ok=True)
    for curr_scan in all_dirs:
        for row, scan in enumerate(df['init_name']): #heuristics file columns
            if scan in curr_scan:
                break 
        convert_nifti(curr_scan,'/home/data/madlab/Mattfeld_vCAT/dset/sub-{0}/'.format(sub)
                      +df['dest_name'][row].format(sub)) 
        print(sub)
        dest_dir = '/home/data/madlab/Mattfeld_vCAT/dset/sub-{0}/'.format(sub)     

### Convert NIFTI to STL for printing 3D Brains

In [ ]:
import os
import os.path as op
import tarfile
import subprocess as sp
from glob import glob
import shutil 

for tar_file in glob('/home/data/madlab/Mattfeld_vCAT/sourcedata/Mattfeld_REVL-000-vCAT-*-S1.tar.gz'):
    sub = os.path.basename(tar_file).split('_')[-1].split('-')[-2] #gets only subject ID
    sp.Popen('mris_convert --combinesurfs /home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer/sub-{0}/surf/lh.pial /home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer/sub-{0}/surf/rh.pial /home/data/madlab/Mattfeld_vCAT/derivatives/stl/sub-{0}_brain.stl'.format(sub), shell=True).wait()
    print(sub)

In [ ]:
%%bash

mris_convert --combinesurfs /home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer/sub-010/surf/lh.pial /home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer/sub-010/surf/rh.pial /home/data/madlab/Mattfeld_vCAT/derivatives/stl/sub-010/sub-010_anat.stl